<a href="https://colab.research.google.com/github/serrabaysal/auto-ml-course-notes/blob/main/27aral%C4%B1k_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Sürüm çakışmalarını önlemek için temiz kurulum
!pip uninstall -q protobuf -y
!pip install -q "protobuf<4.0.0" "flwr[simulation]" torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.1/155.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.3/201.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 39.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not cu

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import flwr as fl
from collections import OrderedDict

import os
# Ray kütüphanesinin simülasyon için düzgün başlatıldığından emin olun
os.environ["RAY_DEDUP_LOGS"] = "0"

import torch
import flwr as fl

# 1. Basit Bir Model Tanımı (AutoML bu yapıyı da optimize edebilir)
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(10, 1) # 10 girişli basit bir regresyon/sınıflandırma

    def forward(self, x):
        return self.fc(x)

# 2. İstemci (Client) Tanımı
# FedHPO burada devreye girer: Sunucu tarafından gönderilen "config" içindeki
# hiperparametreleri kullanarak eğitim yapar.
class AutoMLClient(fl.client.NumPyClient):
    def __init__(self, train_loader):
        self.model = SimpleNet()
        self.train_loader = train_loader

    def get_parameters(self, config):
        return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

    def set_parameters(self, parameters):
        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
        self.model.load_state_dict(state_dict, strict=True)

    def fit(self, parameters, config):
        self.set_parameters(parameters)

        # --- AutoML'in Dokunduğu Yer: Hiperparametre Entegrasyonu ---
        lr = config.get("learning_rate", 0.01) # Sunucudan gelen LR
        optimizer = optim.SGD(self.model.parameters(), lr=lr)

        # Sembolik Eğitim Döngüsü
        for _ in range(1): # 1 local epoch
            for x, y in self.train_loader:
                optimizer.zero_grad()
                loss = nn.MSELoss()(self.model(x), y)
                loss.backward()
                optimizer.step()

        return self.get_parameters(config={}), len(self.train_loader), {}

# 3. Sunucu Tarafı: Hiperparametre Seçimi (Basit Grid Search AutoML)
def on_fit_config_fn(server_round: int):
    # AutoML Stratejisi: Her turda farklı bir öğrenme oranı dene
    # Pratik uygulamada burada Bayesian Opt. veya Optuna çalışır
    learning_rates = [0.1, 0.05, 0.01, 0.001]
    lr = learning_rates[server_round % len(learning_rates)]
    print(f"\n[SUNUCU] Tur {server_round}: AutoML '{lr}' öğrenme oranını seçti.")
    return {"learning_rate": lr}

# 4. Simülasyonu Başlatma
# Sahte veri oluşturma
train_data = [(torch.randn(10), torch.randn(1)) for _ in range(32)]
train_loader = torch.utils.data.DataLoader(train_data, batch_size=8)

# Federatif Simülasyon (2 İstemci ile)
fl.simulation.start_simulation(
    client_fn=lambda cid: AutoMLClient(train_loader),
    num_clients=2,
    config=fl.server.ServerConfig(num_rounds=4),
    strategy=fl.server.strategy.FedAvg(on_fit_config_fn=on_fit_config_fn),
    ray_init_args={"num_cpus": 2, "num_gpus": 0} # Colab kaynak yönetimi
)

INFO flwr 2025-12-27 09:37:19,387 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=4, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=4, round_timeout=None)
2025-12-27 09:37:43,462	INFO worker.py:1998 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
/usr/local/lib/python3.12/dist-packages/ray/_private/worker.py:2046: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(
INFO flwr 2025-12-27 09:37:53,261 | app.py:180 | Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'memory': 9173531853.0, 'CPU': 2.0, 'object_store_memory': 3931513651.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'node:1


[SUNUCU] Tur 1: AutoML '0.05' öğrenme oranını seçti.


(pid=1006) 2025-12-27 09:38:02.965014: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=1006) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=1006) E0000 00:00:1766828283.018897    1006 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=1006) E0000 00:00:1766828283.040046    1006 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=1006) W0000 00:00:1766828283.078366    1006 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
(pid=1006) W0000 00:00:1766828283.078404    1006 computation_placer.cc:177] computation placer already registered. Pleas


[SUNUCU] Tur 2: AutoML '0.01' öğrenme oranını seçti.

[SUNUCU] Tur 3: AutoML '0.001' öğrenme oranını seçti.


DEBUG flwr 2025-12-27 09:38:14,799 | server.py:182 | evaluate_round 3 received 0 results and 2 failures
DEBUG:flwr:evaluate_round 3 received 0 results and 2 failures
DEBUG flwr 2025-12-27 09:38:14,803 | server.py:218 | fit_round 4: strategy sampled 2 clients (out of 2)
DEBUG:flwr:fit_round 4: strategy sampled 2 clients (out of 2)
DEBUG flwr 2025-12-27 09:38:14,904 | server.py:232 | fit_round 4 received 2 results and 0 failures
DEBUG:flwr:fit_round 4 received 2 results and 0 failures
DEBUG flwr 2025-12-27 09:38:14,907 | server.py:168 | evaluate_round 4: strategy sampled 2 clients (out of 2)
DEBUG:flwr:evaluate_round 4: strategy sampled 2 clients (out of 2)
DEBUG flwr 2025-12-27 09:38:14,974 | server.py:182 | evaluate_round 4 received 0 results and 2 failures
DEBUG:flwr:evaluate_round 4 received 0 results and 2 failures
INFO flwr 2025-12-27 09:38:14,976 | server.py:147 | FL finished in 13.280488131999988
INFO:flwr:FL finished in 13.280488131999988
INFO flwr 2025-12-27 09:38:14,978 | app.


[SUNUCU] Tur 4: AutoML '0.1' öğrenme oranını seçti.


In [2]:
import torch
import torch.nn as nn
import flwr as fl
from collections import OrderedDict

# 1. Dinamik Mimari: AutoML katman genişliğini (width) değiştirecek
class DynamicNet(nn.Module):
    def __init__(self, hidden_width=16):
        super().__init__()
        self.fc1 = nn.Linear(10, hidden_width) # Giriş katmanı -> Değişken gizli katman
        self.fc2 = nn.Linear(hidden_width, 1)  # Gizli katman -> Çıkış

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

# 2. AutoML İstemcisi
class FedNASClient(fl.client.NumPyClient):
    def __init__(self, train_loader):
        self.train_loader = train_loader
        self.model = None # Model her turda yeniden yapılandırılacak

    def fit(self, parameters, config):
        # AutoML'in seçtiği mimari genişliğini al
        width = config.get("hidden_width", 16)
        self.model = DynamicNet(hidden_width=width)

        # Ağırlıkları yükle (Eğer mimari değiştiyse sadece uyumlu kısımları yükler)
        # Basitlik adına bu örnekte her mimari adayını sıfırdan eğitiyoruz

        optimizer = torch.optim.SGD(self.model.parameters(), lr=0.01)

        # Eğitim ve Kayıp Hesaplama
        total_loss = 0.0
        for x, y in self.train_loader:
            optimizer.zero_grad()
            output = self.model(x)
            loss = torch.nn.MSELoss()(output, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f" -> İstemci: Mimari Genişliği {width} ile eğitildi. Kayıp: {total_loss:.4f}")

        # Yeni ağırlıkları ve eğitim örnek sayısını döndür
        params = [val.cpu().numpy() for _, val in self.model.state_dict().items()]
        return params, len(self.train_loader), {"loss": total_loss}

# 3. Sunucu Stratejisi: Mimari Arama
def on_fit_config_fn(server_round: int):
    # AutoML Arama Uzayı: Farklı nöron sayıları
    architectures = [8, 32, 128, 512]
    selected_width = architectures[server_round % len(architectures)]

    print(f"\n[AutoML SUNUCU] Tur {server_round+1}: Test Edilen Mimari Genişliği = {selected_width} Nöron")
    return {"hidden_width": selected_width}

# 4. Simülasyon Başlatma
train_data = [(torch.randn(10), torch.randn(1)) for _ in range(64)]
train_loader = torch.utils.data.DataLoader(train_data, batch_size=16)

fl.simulation.start_simulation(
    client_fn=lambda cid: FedNASClient(train_loader),
    num_clients=3,
    config=fl.server.ServerConfig(num_rounds=4),
    strategy=fl.server.strategy.FedAvg(on_fit_config_fn=on_fit_config_fn),
    client_resources={"num_cpus": 1, "num_gpus": 0}
)

INFO flwr 2025-12-27 09:39:27,253 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=4, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=4, round_timeout=None)
2025-12-27 09:39:35,753	INFO worker.py:2007 -- Started a local Ray instance.
INFO flwr 2025-12-27 09:39:44,996 | app.py:180 | Flower VCE: Ray initialized with resources: {'object_store_memory': 3932499148.0, 'CPU': 2.0, 'node:__internal_head__': 1.0, 'GPU': 1.0, 'memory': 9175831348.0, 'accelerator_type:T4': 1.0, 'node:172.28.0.12': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'object_store_memory': 3932499148.0, 'CPU': 2.0, 'node:__internal_head__': 1.0, 'GPU': 1.0, 'memory': 9175831348.0, 'accelerator_type:T4': 1.0, 'node:172.28.0.12': 1.0}
INFO flwr 2025-12-27 09:39:44,997 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2025-12-27 09:39:45,003 | server.py:273 | Requesting initial parameters from o


[AutoML SUNUCU] Tur 2: Test Edilen Mimari Genişliği = 32 Nöron


(pid=1828) 2025-12-27 09:39:54.304746: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=1828) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=1828) E0000 00:00:1766828394.371431    1828 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=1828) E0000 00:00:1766828394.391245    1828 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=1828) W0000 00:00:1766828394.437947    1828 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
(pid=1828) W0000 00:00:1766828394.437978    1828 computation_placer.cc:177] computation placer already registered. Pleas

(launch_and_fit pid=1829)  -> İstemci: Mimari Genişliği 32 ile eğitildi. Kayıp: 5.3341
(launch_and_fit pid=1829)  -> İstemci: Mimari Genişliği 32 ile eğitildi. Kayıp: 5.5718


DEBUG flwr 2025-12-27 09:40:04,160 | server.py:232 | fit_round 1 received 3 results and 0 failures
DEBUG:flwr:fit_round 1 received 3 results and 0 failures
WARNING flwr 2025-12-27 09:40:04,164 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2025-12-27 09:40:04,166 | server.py:168 | evaluate_round 1: strategy sampled 3 clients (out of 3)
DEBUG:flwr:evaluate_round 1: strategy sampled 3 clients (out of 3)
DEBUG flwr 2025-12-27 09:40:04,259 | server.py:182 | evaluate_round 1 received 0 results and 3 failures
DEBUG:flwr:evaluate_round 1 received 0 results and 3 failures
DEBUG flwr 2025-12-27 09:40:04,260 | server.py:218 | fit_round 2: strategy sampled 3 clients (out of 3)
DEBUG:flwr:fit_round 2: strategy sampled 3 clients (out of 3)
DEBUG flwr 2025-12-27 09:40:04,356 | server.py:232 | fit_round 2 received 3 results and 0 failures
DEBUG:flwr:fit_round 2 received 3 results and 0 failures
DEBUG flwr 2025-12-27 09:40:04,360 | server.py:168 | evaluate_round 2: strategy sample

(launch_and_fit pid=1828)  -> İstemci: Mimari Genişliği 32 ile eğitildi. Kayıp: 5.9492

[AutoML SUNUCU] Tur 3: Test Edilen Mimari Genişliği = 128 Nöron
(launch_and_fit pid=1829)  -> İstemci: Mimari Genişliği 128 ile eğitildi. Kayıp: 5.5765


DEBUG flwr 2025-12-27 09:40:04,452 | server.py:182 | evaluate_round 2 received 0 results and 3 failures
DEBUG:flwr:evaluate_round 2 received 0 results and 3 failures
DEBUG flwr 2025-12-27 09:40:04,453 | server.py:218 | fit_round 3: strategy sampled 3 clients (out of 3)
DEBUG:flwr:fit_round 3: strategy sampled 3 clients (out of 3)
DEBUG flwr 2025-12-27 09:40:04,553 | server.py:232 | fit_round 3 received 3 results and 0 failures
DEBUG:flwr:fit_round 3 received 3 results and 0 failures
DEBUG flwr 2025-12-27 09:40:04,556 | server.py:168 | evaluate_round 3: strategy sampled 3 clients (out of 3)
DEBUG:flwr:evaluate_round 3: strategy sampled 3 clients (out of 3)
DEBUG flwr 2025-12-27 09:40:04,641 | server.py:182 | evaluate_round 3 received 0 results and 3 failures
DEBUG:flwr:evaluate_round 3 received 0 results and 3 failures
DEBUG flwr 2025-12-27 09:40:04,642 | server.py:218 | fit_round 4: strategy sampled 3 clients (out of 3)
DEBUG:flwr:fit_round 4: strategy sampled 3 clients (out of 3)



[AutoML SUNUCU] Tur 4: Test Edilen Mimari Genişliği = 512 Nöron

[AutoML SUNUCU] Tur 5: Test Edilen Mimari Genişliği = 8 Nöron


DEBUG flwr 2025-12-27 09:40:04,743 | server.py:232 | fit_round 4 received 3 results and 0 failures
DEBUG:flwr:fit_round 4 received 3 results and 0 failures
DEBUG flwr 2025-12-27 09:40:04,746 | server.py:168 | evaluate_round 4: strategy sampled 3 clients (out of 3)
DEBUG:flwr:evaluate_round 4: strategy sampled 3 clients (out of 3)
DEBUG flwr 2025-12-27 09:40:04,832 | server.py:182 | evaluate_round 4 received 0 results and 3 failures
DEBUG:flwr:evaluate_round 4 received 0 results and 3 failures
INFO flwr 2025-12-27 09:40:04,834 | server.py:147 | FL finished in 11.535586038999952
INFO:flwr:FL finished in 11.535586038999952
INFO flwr 2025-12-27 09:40:04,838 | app.py:218 | app_fit: losses_distributed []
INFO:flwr:app_fit: losses_distributed []
INFO flwr 2025-12-27 09:40:04,839 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2025-12-27 09:40:04,841 | app.py:220 | app_fit: metrics_distributed {}
INFO:flwr:app_fit: metrics_distributed 